In [ ]:
# make sure we are working in module directory
repo_root = !git rev-parse --show-toplevel
module_path = repo_root[0] + "/backend/heatflask"
print("Working directory")
%cd $module_path

In [ ]:
# %load Strava.py
"""
***  For Jupyter notebook ***

Paste one of these Jupyter magic directives to the top of a cell
 and run it, to do these things:

  * %%cython --annotate
      Compile and run the cell

  * %load Strava.py
     Load Strava.py file into this (empty) cell

  * %%writefile Strava.py
      Write the contents of this cell to Strava.py

"""

import os
import time
import aiohttp
from logging import getLogger
import urllib
import msgpack
import polyline
import asyncio
import datetime

import StreamCodecs

log = getLogger(__name__)
log.propagate = True


STRAVA_DOMAIN = "https://www.strava.com"
STALE_TOKEN = 300

# Client class takes care of refreshing access tokens
class AsyncClient:
    def __init__(self, name, refresh_callback=None, **kwargs):
        self.name = name
        self.set_state(**kwargs)
        self.refresh_callback = refresh_callback

    def set_state(
        self, access_token=None, expires_at=None, refresh_token=None, **extra
    ):
        self.access_token = access_token
        self.expires_at = int(expires_at)
        self.refresh_token = refresh_token
        self.session = self.new_session()

    def __repr__(self):
        expires_at_str = datetime.datetime.fromtimestamp(self.expires_at)
        return f"<AsyncClient '{self.name}' expires-{expires_at_str}>"

    @property
    def expires_in(self):
        return self.expires_at - round(time.time())

    @property
    def headers(self):
        if self.access_token:
            return {"Authorization": f"Bearer {self.access_token}"}

    def new_session(self):
        return aiohttp.ClientSession(
            STRAVA_DOMAIN, headers=self.headers, raise_for_status=True
        )

    async def update_access_token(self, stale_ttl=STALE_TOKEN, code=None):
        if not (self.refresh_token or code):
            return

        if (code is None) and (self.expires_in > stale_ttl):
            log.debug("%s is valid for %ss", self, self.expires_in)
            return

        log.debug("%s updating access token", self)
        t0 = time.perf_counter()
        new_auth_info = await self.run_func(
            get_access_token, code=code, refresh_token=self.refresh_token
        )

        if not (new_auth_info and new_auth_info.get("refresh_token")):
            return

        await self.session.close()
        self.set_state(**new_auth_info)

        if self.refresh_callback:
            await self.refresh_callback(self.name, new_auth_info)

        elapsed = time.perf_counter() - t0
        log.info("%s token refresh took %.2f", self.name, elapsed)

        return new_auth_info

    async def run_func(self, func, *args, **kwargs):
        try:
            return await func(self.session, *args, **kwargs)
        except Exception:
            log.exception("%s, %s", self, func)

    def get_athlete(self):
        return self.run_func(get_athlete)

    def get_streams(self, activity_id):
        return self.run_func(get_streams, activity_id)

    def get_activity(self, activity_id):
        return self.run_func(get_activity, activity_id)
    
    def get_index(self):
        return get_index(self.session)


# *********************************************************************************************
API_SPEC = "/api/v3"

#
# Authentication
#

AUTH_ENDPOINT = "/oauth/authorize"
AUTH_PARAMS = {
    "client_id": os.environ["STRAVA_CLIENT_ID"],
    "response_type": "code",
    "approval_prompt": "auto",  # or "force"
    "scope": "read,activity:read,activity:read_all",
    "redirect_uri": None,
    "state": None,
}

TOKEN_EXCHANGE_ENDPOINT = "/oauth/token"
TOKEN_EXCHANGE_PARAMS = {
    "client_id": os.environ["STRAVA_CLIENT_ID"],
    "client_secret": os.environ["STRAVA_CLIENT_SECRET"],
    #     "code": None,
    #     "grant_type": "authorization_code",
}


def auth_url(redirect_uri=None, state=None):
    params = {**AUTH_PARAMS, "redirect_uri": redirect_uri, "state": state}
    return STRAVA_DOMAIN + AUTH_ENDPOINT + "?" + urllib.parse.urlencode(params)


# We can get the access_token for a user either with
# a code obtained via authentication, or with a refresh token
async def get_access_token(session=None, code=None, refresh_token=None):
    params = {**TOKEN_EXCHANGE_PARAMS}
    params.update(
        {"grant_type": "authorization_code", "code": code}
        if code
        else {"grant_type": "refresh_token", "refresh_token": refresh_token}
    )
    async with session.post(TOKEN_EXCHANGE_ENDPOINT, params=params) as response:
        rjson = await response.json()
    return rjson


#
# Athlete
#
ATHLETE_ENDPOINT = f"{API_SPEC}/athlete"


async def get_athlete(session):
    async with session.get(ATHLETE_ENDPOINT) as response:
        return await response.json()


#
# Streams
#
MIN_STREAM_LENGTH = 3
POLYLINE_PRECISION = 6
ACTIVITY_STREAM_PARAMS = {
    "keys": "latlng,altitude,time",
    "key_by_type": "true",
    "series_type": "time",
    "resolution": "high",
}


def streams_endpoint(activity_id):
    return f"{API_SPEC}/activities/{activity_id}/streams"


async def get_streams(session, activity_id):
    t0 = time.perf_counter()
    async with session.get(
        streams_endpoint(activity_id), params=ACTIVITY_STREAM_PARAMS
    ) as response:
        rjson = await response.json()

        if not (
            rjson and ("time" in rjson) and (len(rjson["time"] < MIN_STREAM_LENGTH))
        ):
            return

        result = msgpack.packb(
            {
                "t": StreamCodecs.rlld_encode(rjson["time"]["data"]),
                "a": StreamCodecs.rlld_encode(rjson["altitude"]["data"]),
                "p": polyline.encode(rjson["latlng"]["data"], POLYLINE_PRECISION),
            }
        )
    elapsed = time.perf_counter() - t0
    log.info("fetching streams for %d took %.1f", activity_id, elapsed)

    return activity_id, result


def unpack_streams(packed_streams):
    streams = msgpack.unpackb(packed_streams)
    return {
        "time": StreamCodecs.rlld_decode(streams["t"], dtype="u2"),
        "altitude": StreamCodecs.rlld_decode(streams["a"], dtype="i2"),
        "latlng": polyline.decode(streams["p"], POLYLINE_PRECISION),
    }


async def get_many_streams(session, activity_ids):
    request_tasks = [
        asyncio.create_task(get_streams(session, aid)) for aid in activity_ids
    ]
    for task in asyncio.as_completed(request_tasks):
        activity_id, status, streams = await task
        if status != 200:
            abort_signal = yield activity_id, streams

        if abort_signal or (status is None):
            log.info("get_many_streams aborted")
            for other_task in request_tasks:
                other_task.cancel()
            await asyncio.wait(request_tasks)
            break


#
# Index pages
#
PER_PAGE = 200
REQUEST_DELAY = 0.2
ACTIVITY_LIST_ENDPOINT = f"{API_SPEC}/athlete/activities"
params = {"per_page": PER_PAGE}


async def get_activity_index_page(session, p):
    t0 = time.perf_counter()

    async with session.get(ACTIVITY_LIST_ENDPOINT, params={**params, "page": p}) as r:
        result = await r.json()

    elapsed_ms = round((time.perf_counter() - t0) * 1000)
    log.debug("retrieved page %d in %d", p, elapsed_ms)
    return p, result


def page_request(session, p):
    return asyncio.create_task(get_activity_index_page(session, p))


async def get_index(user_session):

    done_adding_pages = False
    tasks = set([page_request(user_session, 1)])
    next_page = 2
    while tasks:

        if not done_adding_pages:
            # Add a page request task
            tasks.add(page_request(user_session, next_page))
            log.debug("requesting page %d", next_page)
            next_page += 1

        # wait for a moment to check for any completed requests
        finished, unfinished = await asyncio.wait(
            tasks, return_when=asyncio.FIRST_COMPLETED, timeout=REQUEST_DELAY
        )

        for task in finished:
            p, entries = task.result()

            if len(entries):
                for A in entries:
                    abort_signal = yield A
                    
                    if abort_signal:
                        log.debug("get_index aborted")
                        for task in unfinished:
                            task.cancel()
                        await asyncio.wait(unfinished)
                        return
                    
                log.debug("processed %d entries from page %d", len(entries), p)
            elif not done_adding_pages:
                done_adding_pages = True
                log.debug("Done requesting pages (status %d)", status)
        tasks = unfinished


def activity_endpoint(activity_id):
    return f"{API_SPEC}/activities/{activity_id}?include_all_efforts=false"


async def get_activity(user_session, activity_id):
    async with user_session.get(activity_endpoint(activity_id)) as r:
        return await r.json()


#
# Updates (Webhook subscription)
#
SUBSCRIPTION_VERIFY_TOKEN = "heatflask_yay!"
SUBSCRIPTION_ENDPOINT = f"{API_SPEC}/push_subscriptions"
SUBSCRIPTION_PARAMS = {
    "client_id": os.environ["STRAVA_CLIENT_ID"],
    "client_secret": os.environ["STRAVA_CLIENT_SECRET"],
}
CREATE_SUBSCRIPTION_PARAMS = {
    **SUBSCRIPTION_PARAMS,
    "verify_token": SUBSCRIPTION_VERIFY_TOKEN,
    "callback_url": None,
}
VIEW_SUBSCRIPTION_PARAMS = SUBSCRIPTION_PARAMS
DELETE_SUBSCRIPTION_PARAMS = {
    **SUBSCRIPTION_PARAMS,
    "id": None,
}


async def create_subscription(admin_session, callback_url):
    params = {**CREATE_SUBSCRIPTION_PARAMS, "callback_url": callback_url}
    async with admin_session.post(SUBSCRIPTION_ENDPOINT, params=params) as response:
        return await response.json()


# After calling create_subscription, you will receive a GET request at your
# supplied callback_url, whose json body is validation_dict.
#
# Your response must have HTTP code 200 and be of application/json content type.
# and be the return value of this function.
async def verify_subscription(validation_dict):
    if validation_dict.get("hub.verify_token") != SUBSCRIPTION_VERIFY_TOKEN:
        return {"hub.challenge": validation_dict["hub.challenge"]}


async def view_subscription(admin_session):
    async with admin_session.get(SUBSCRIPTION_ENDPOINT, params=params) as response:
        return await response.json()


async def delete_subscription(admin_session, subscription_id=None):
    params = {**DELETE_SUBSCRIPTION_PARAMS, "id": subscription_id}
    async with admin_session.delete(SUBSCRIPTION_ENDPOINT, params=params) as response:
        return response.status == 204


In [ ]:
# Test Create Client

import aiofiles
import json

log.setLevel("DEBUG")

credentials_filename = "__pycache__/test-credentials.json"
async with aiofiles.open(credentials_filename, mode='r') as f:
    contents = await f.read()
    access_info = json.loads(contents)

    
async def update_credentials_file(client_name, access_info):
    async with aiofiles.open(credentials_filename, 'w') as json_file:
        await json_file.write(json.dumps(access_info, indent=2))
    log.info("new credentials saved to %s: %s", credentials_filename, access_info)
    
C = AsyncClient("erensi", **access_info, refresh_callback=update_credentials_file)
C, C.expires_in

In [ ]:
# Test refresh access_token

await C.update_access_token()

# Force refresh token
await C.update_access_token(stale_ttl=C.expires_in + 1)

In [ ]:
gen = C.get_index()

In [ ]:
await gen.__anext__()

In [ ]:
await C.get_streams(122445)

In [ ]:


activities = []
async with aiohttp.ClientSession(STRAVA_DOMAIN, headers=HEADERS) as session:
    generator = get_index(session)
    async for a in generator:
        activities.append(a)
    

In [ ]:
unpack_streams(result[6611297682])

In [ ]:
not []